In [ ]:
import pandas as pd

data_filename = 'winter_wheat_yield.csv'

weather_dir = './WEATHER-DATA--v01/'
wdtemplate = r'weather-data-for-index__{padded}.csv'

total_data = pd.read_csv('./winter_wheat_yield.csv')
print(total_data.shape)
print()

w_df = {}
for i in range(0,len(total_data)):
    padded = str(i).zfill(4)
    w_df[i] = pd.read_csv(weather_dir + wdtemplate.format(padded=padded))
    # Want to have a name for the index of my dataframe
    w_df[i].rename(columns={'Unnamed: 0': 'date'}, 
                   inplace=True)
    # w_df[i] = w_df[i].rename_axis(index='DATE')
    
print()
print(w_df[4].shape)
print(w_df[4].head())

In [ ]:
def create_weekly_df(df):
    df1 = df.copy()
    # convert index to datetime format    
    df1.index = pd.to_datetime(df['date'], format='%Y%m%d')
    # use 'M' for monthly, use 'W' for weekly
    df1_weekly = df1.resample('W').agg({'T2M_MAX':'mean',
                                       'T2M_MIN':'mean',
                                       'PRECTOTCORR':'sum',
                                       'GWETROOT':'mean',
                                       'EVPTRNS':'mean',
                                       'ALLSKY_SFC_PAR_TOT':'sum',
                                       'HDD10':'mean',
                                       'SNODP':'mean'})    

    # convert index back to string format YYYYMM
    df1_weekly.index = df1_weekly.index.strftime('%Y%m%d')
    
    return df1_weekly


print(create_weekly_df(w_df[4]).head(50))

In [ ]:
import json

df_t0 = w_df[4]
cols_narrow = df_t0.columns.values.tolist()[1:]
print(cols_narrow)

print()

df_t1 = create_weekly_df(df_t0)     # dfw['0001']
# print(df_t1.head())

cols_wide = []
for i in range(0,len(df_t1)):
    row = df_t1.iloc[i]
    # print(row)
    # can't use date, because it has year built in, and weeks start on different numbers...
    week_id = 'week_' + str(i).zfill(2)
    # print(date)
    for c in cols_narrow:
        cols_wide.append(week_id + '__' + c)
        
print(cols_wide)
print(len(cols_wide))

In [ ]:
print(w_df[0].columns.tolist()[1:])
print(w_df[0].shape)
print(create_weekly_df(w_df[0]).shape)

def create_weather_seq_for_weekly(dfw):
    seq = []
    for i in range(0,len(dfw)):
        for c in cols_narrow:
            seq.append(dfw.iloc[i][c])
    return seq

# sanity check
dfw = create_weekly_df(w_df[0])
print(dfw.head(10))

seqw = create_weather_seq_for_weekly(dfw)
print(json.dumps(seqw, indent=4))

In [ ]:
import time

u_df = {}
dfw = {}
seqw = {}


for i in range(0,len(total_data)):
    padded = str(i).zfill(4)
    # print(padded)
    u_df[padded] = pd.read_csv(weather_dir + wdtemplate.format(padded=padded))
    # Want to have a name for the index of my dataframe
    u_df[padded].rename(columns={'Unnamed: 0': 'date'}, 
                   inplace=True)
    
    dfw[padded] = create_weekly_df(u_df[padded])
    # print(dfw.head())

    seqw[i] = create_weather_seq_for_weekly(dfw[padded])
    # print(json.dumps(dictw, indent=4)

    # introducing a small occassional sleep because my python kernel kept complaining about
    # exceeding some I/O threshold
    # if i % 30 == 0:
    #     time.sleep(0.05)
        
    # if i > 4000 and i % 100 == 0:
    #     time.sleep(0.5)
    
    if i % 100 == 0:
        print('Completed processing of index ', i)
    
# sanity check
print(print(json.dumps(seqw, indent=4)))

In [ ]:

#for i in seqw:
#    if len(seqw[i]) != 248:
#        print(i)
#        print(seqw[i])


#print(len(seqw[15]))
#print(dfw['0000'].shape)
print(len(cols_wide))
print(len(total_data))
print(len(seqw[0]))
print()

df_wide_weather_weekly_prelim = pd.DataFrame.from_dict(seqw, orient='index', columns=cols_wide)

print(df_wide_weather_weekly_prelim.shape)
print()
print(df_wide_weather_weekly_prelim.head())


In [ ]:
week_31_cols = ['week_31__T2M_MAX', 'week_31__T2M_MIN', 'week_31__PRECTOTCORR', 'week_31__GWETROOT', 'week_31__EVPTRNS', 'week_31__ALLSKY_SFC_PAR_TOT', 'week_31__HDD10', 'week_31__SNODP']

df_wide_weather_weekly = df_wide_weather_weekly_prelim.drop(columns=week_31_cols)

print()
print(df_wide_weather_weekly.shape)
print(df_wide_weather_weekly.head())

In [ ]:
print(df_wide_weather_weekly)
print(total_data)

In [ ]:
weather_yield_data = pd.concat([total_data, df_wide_weather_weekly], axis='columns')

print(weather_yield_data)

weather_yield_data.to_csv("./weekly_weather_data.csv", index=False)